In [10]:
pip install --upgrade rembg pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install onnxruntime

In [2]:
pip install rembg

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox, Canvas, Scale
from rembg import remove, new_session
from PIL import Image, ImageTk, ImageDraw
import io
import threading

class BackgroundRemoverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Background Remover")

        # Initialize variables
        self.original_image = None
        self.processed_image = None
        self.draw_image = None
        self.zoom_factor = 1.0  # Zoom level
        self.min_zoom = 0.5
        self.max_zoom = 3.0

        # Preload rembg model for fast processing
        self.session = new_session(providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

        # Create UI
        self.create_widgets()

    def create_widgets(self):
        self.canvas = Canvas(self.root, width=400, height=400, bg="white")
        self.canvas.pack(pady=10)

        self.upload_button = tk.Button(self.root, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(pady=5)

        self.remove_button = tk.Button(self.root, text="Remove Background", command=self.remove_background)
        self.remove_button.pack(pady=5)

        self.save_button = tk.Button(self.root, text="Download Processed Image", command=self.save_image)
        self.save_button.pack(pady=5)

        # Bind mouse wheel zoom
        self.canvas.bind("<MouseWheel>", self.zoom_image)

    def upload_image(self):
        file_path = filedialog.askopenfilename(
            title="Select an Image",
            filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
        )
        if file_path:
            self.load_image(file_path)

    def load_image(self, file_path):
        try:
            self.original_image = Image.open(file_path).convert("RGBA")
            self.processed_image = self.original_image.copy()
            self.draw_image = self.original_image.copy()
            self.zoom_factor = 1.0  # Reset zoom on new image

            self.show_image(self.draw_image)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def show_image(self, image):
        """Display image on the canvas with current zoom level."""
        width, height = image.size
        new_size = (int(width * self.zoom_factor), int(height * self.zoom_factor))
        resized_image = image.resize(new_size, Image.Resampling.LANCZOS)

        self.tk_image = ImageTk.PhotoImage(resized_image)
        self.canvas.delete("all")  # Clear previous images
        self.canvas.create_image(200, 200, image=self.tk_image)

    def remove_background(self):
        if self.original_image:
            threading.Thread(target=self.process_background_removal, daemon=True).start()

    def process_background_removal(self):
        try:
            resized_image = self.original_image.copy()
            resized_image.thumbnail((800, 800))

            input_image = io.BytesIO()
            resized_image.save(input_image, format='PNG')

            output_image = remove(input_image.getvalue(), session=self.session)

            self.processed_image = Image.open(io.BytesIO(output_image))
            self.draw_image = self.processed_image.copy()

            self.show_image(self.draw_image)
            messagebox.showinfo("Success", "Background removed successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def zoom_image(self, event):
        """Zoom in/out with mouse wheel."""
        if event.delta > 0 and self.zoom_factor < self.max_zoom:
            self.zoom_factor *= 1.1  # Zoom in
        elif event.delta < 0 and self.zoom_factor > self.min_zoom:
            self.zoom_factor /= 1.1  # Zoom out
        
        self.show_image(self.draw_image)

    def save_image(self):
        if self.draw_image:
            output_path = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[("PNG files", "*.png")]
            )
            if output_path:
                self.draw_image.save(output_path)
                messagebox.showinfo("Success", "Image saved successfully!")

# Create the main window
root = tk.Tk()
app = BackgroundRemoverApp(root)

# Run the application
root.mainloop()